### Sử dụng DASK để đọc file Sản lượng có nhiều Sheets

In [3]:
import dask.dataframe as dd
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Các chị thay đường dẫn file sản lượng vào phần này ạ
file_path = r"C:\Khue\TDN\data\raw\ipp-sanluong-2024.xlsx"
xls = pd.ExcelFile(file_path)
sheet_names = xls.sheet_names

# Xử lý mỗi sheet và kết hợp chúng thành 1 DataFrame
dfs = []
for sheet in sheet_names:
    # Đọc sheet thành pandas DataFrame
    df = pd.read_excel(file_path, sheet_name=sheet)
    # Chuyển đổi thành Dask DataFrame
    ddf = dd.from_pandas(df, npartitions=10)  # Chia thành 10 phần
    dfs.append(ddf)

# Ghép tất cả Dask DataFrame
combined_df = dd.concat(dfs)

In [4]:
# Chuyển đổi kết quả từ dask DataFrame sang pandas DataFrame
result_df = combined_df.compute()
# reset index
result_df.reset_index(drop=True, inplace=True)
result_df

,CTDL,NMTD,MADIEMDO,TENDIEMDO,SOCONGTO,STARTTIME,ENDTIME,CS,SL_PGIAOTONG
0,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ TÀ LÀNG,G2A121S000M371,Công tơ Chính - 371,16164184,2024-08-07 23:30:00,2024-08-07 23:30:00,3395.8,0.0
1,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ TÀ LÀNG,G2A121S000M371,Công tơ Chính - 371,16164184,2024-08-07 18:30:00,2024-08-07 18:30:00,2984.8,0.0
2,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ TÀ LÀNG,G2A121S000M371,Công tơ Chính - 371,16164184,2024-08-07 19:00:00,2024-08-07 19:00:00,2363.7,0.0
3,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ TÀ LÀNG,G2A121S000M371,Công tơ Chính - 371,16164184,2024-08-08 07:30:00,2024-08-08 07:30:00,3325.8,0.0
4,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ TÀ LÀNG,G2A121S000M371,Công tơ Chính - 371,16164184,2024-08-08 08:00:00,2024-08-08 08:00:00,3294.5,0.0
...,...,...,...,...,...,...,...,...,...
3577158,CTY ĐIỆN LỰC TUYÊN QUANG,NMTĐ Sông Lô 8B,G2A214S000M131,Sông Lô 8B_131c,40774734,2024-08-07 01:30:00,2024-08-07 01:30:00,0.0,241770163.0
3577159,CTY ĐIỆN LỰC TUYÊN QUANG,NMTĐ Sông Lô 8B,G2A214S000M131,Sông Lô 8B_131c,40774734,2024-08-07 02:00:00,2024-08-07 02:00:00,0.0,241770163.0
3577160,CTY ĐIỆN LỰC TUYÊN QUANG,NMTĐ Sông Lô 8B,G2A214S000M131,Sông Lô 8B_131c,40774734,2024-08-07 05:30:00,2024-08-07 05:30:00,0.0,241770163.0
3577161,CTY ĐIỆN LỰC TUYÊN QUANG,NMTĐ Sông Lô 8B,G2A214S000M131,Sông Lô 8B_131c,40774734,2024-08-07 06:00:00,2024-08-07 06:00:00,17.0,241770973.1


In [5]:
# chỉ giữ lại các cột MADIEMDO, STARTTIME, CS của result_df
df = result_df[["MADIEMDO", "STARTTIME", "CS"]]
df

,MADIEMDO,STARTTIME,CS
0,G2A121S000M371,2024-08-07 23:30:00,3395.8
1,G2A121S000M371,2024-08-07 18:30:00,2984.8
2,G2A121S000M371,2024-08-07 19:00:00,2363.7
3,G2A121S000M371,2024-08-08 07:30:00,3325.8
4,G2A121S000M371,2024-08-08 08:00:00,3294.5
...,...,...,...
3577158,G2A214S000M131,2024-08-07 01:30:00,0.0
3577159,G2A214S000M131,2024-08-07 02:00:00,0.0
3577160,G2A214S000M131,2024-08-07 05:30:00,0.0
3577161,G2A214S000M131,2024-08-07 06:00:00,17.0


In [6]:
df["NGAY"] = df["STARTTIME"].dt.date
df["CHU_KY"] = df["STARTTIME"].dt.time
df = df.drop(columns=["STARTTIME"])
df

,MADIEMDO,CS,NGAY,CHU_KY
0,G2A121S000M371,3395.8,2024-08-07,23:30:00
1,G2A121S000M371,2984.8,2024-08-07,18:30:00
2,G2A121S000M371,2363.7,2024-08-07,19:00:00
3,G2A121S000M371,3325.8,2024-08-08,07:30:00
4,G2A121S000M371,3294.5,2024-08-08,08:00:00
...,...,...,...,...
3577158,G2A214S000M131,0.0,2024-08-07,01:30:00
3577159,G2A214S000M131,0.0,2024-08-07,02:00:00
3577160,G2A214S000M131,0.0,2024-08-07,05:30:00
3577161,G2A214S000M131,17.0,2024-08-07,06:00:00


In [7]:
# Convert CHU_KY to datetime format first
df["CHU_KY"] = pd.to_datetime(df["CHU_KY"].astype(str)).dt.time

# Filter rows where minutes are multiples of 30 and second is always = 00
df = df[
    (pd.to_datetime(df["CHU_KY"].astype(str)).dt.minute % 30 == 0)
    & (pd.to_datetime(df["CHU_KY"].astype(str)).dt.second == 0)
]
df


,MADIEMDO,CS,NGAY,CHU_KY
0,G2A121S000M371,3395.8,2024-08-07,23:30:00
1,G2A121S000M371,2984.8,2024-08-07,18:30:00
2,G2A121S000M371,2363.7,2024-08-07,19:00:00
3,G2A121S000M371,3325.8,2024-08-08,07:30:00
4,G2A121S000M371,3294.5,2024-08-08,08:00:00
...,...,...,...,...
3577158,G2A214S000M131,0.0,2024-08-07,01:30:00
3577159,G2A214S000M131,0.0,2024-08-07,02:00:00
3577160,G2A214S000M131,0.0,2024-08-07,05:30:00
3577161,G2A214S000M131,17.0,2024-08-07,06:00:00


In [ ]:
# Chuyển file sản lượng thành dạng chia theo ngày và 48 chu kỳ
new_data = pd.pivot_table(
    df, values="CS", index=["MADIEMDO", "NGAY"], columns="CHU_KY"
).reset_index()
new_data


CHU_KY,MADIEMDO,NGAY,00:00:00,00:30:00,01:00:00,01:30:00,02:00:00,02:30:00,03:00:00,03:30:00,...,19:00:00,19:30:00,20:00:00,20:30:00,21:00:00,21:30:00,22:00:00,22:30:00,23:00:00,23:30:00
0,15098879,2024-05-12,0.000,0.000,0.000,0.00,0.000,0.00,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,15098879,2024-05-13,0.000,0.000,0.000,0.00,0.000,0.00,0.000,0.000,...,1401.100,793.260,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,15098879,2024-05-14,0.000,0.000,0.000,0.00,0.000,0.00,0.000,0.000,...,1660.100,1062.300,0.080,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,15098879,2024-05-15,0.000,0.000,0.000,0.00,0.000,0.00,0.000,0.000,...,1229.200,853.500,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4,15098879,2024-05-16,0.000,0.000,0.000,0.00,0.000,0.00,0.000,0.000,...,1971.800,1951.900,1733.200,1593.700,1564.400,1362.500,1319.600,1282.100,1062.100,27.160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74987,G2A298S000M131,2024-08-02,0.084,0.011,0.000,0.00,0.000,0.00,0.000,0.000,...,0.000,0.116,0.123,0.109,0.077,0.058,0.000,0.006,0.072,0.000
74988,G2A298S000M131,2024-08-03,0.000,0.000,0.000,0.00,0.000,0.00,0.000,0.000,...,0.168,0.185,0.159,0.081,0.000,0.000,0.000,0.000,0.000,0.000
74989,G2A298S000M131,2024-08-04,0.000,0.000,0.000,0.00,0.000,0.00,0.000,0.000,...,0.000,0.026,0.000,0.025,0.151,0.028,0.097,0.140,0.147,0.000
74990,G2A298S000M131,2024-08-05,0.000,0.000,0.000,0.00,0.000,0.00,0.000,0.000,...,0.287,0.240,0.000,0.000,0.067,0.270,0.024,0.107,0.000,0.005


In [47]:
# Các chị để đường dẫn file thông tin NMTD
file_path_2 = r"C:\Khue\TDN\data\raw\thong_tin_NMTD_convert.xlsx"
df_NMTD = pd.read_excel(file_path_2)
df_NMTD

,DVDL,NMTD,TEN_NM,MADIEMDO
0,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ TÀ LÀNG,TA_LANG,G2A121S000D371
1,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ TÀ LÀNG,TA_LANG,G2A121S000M371
2,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ THƯỢNG ÂN,THUONG_AN,G2A122S000D371
3,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ THƯỢNG ÂN,THUONG_AN,G2A122S000M371
4,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ NẬM CẮT,NAM_CAT,G2A123S000M371
...,...,...,...,...
802,CTY ĐIỆN LỰC TUYÊN QUANG,NMTĐ Sông Lô 8A,SONG_LO_8A,G2A203S000D131
803,CTY ĐIỆN LỰC TUYÊN QUANG,NMTĐ Sông Lô 8B,SONG_LO_8B,G2A214S000D132
804,CTY ĐIỆN LỰC TUYÊN QUANG,NMTĐ Sông Lô 8B,SONG_LO_8B,G2A214S000M132
805,CTY ĐIỆN LỰC TUYÊN QUANG,NMTĐ Sông Lô 8B,SONG_LO_8B,G2A214S000D131


In [ ]:
# nCác chị để đường dẫn file DS_TTT_TDN
df_TDN = pd.read_excel(r"C:\Khue\TDN\data\raw\DS_TTT_TDN.xlsx")

In [ ]:
# Merge TD_THAMCHIEU cho file thông tin NMTD bằng cột key là 'TEN_NM'
merged_df = df_NMTD.merge(df_TDN[["TEN_NM", "TD_THAMCHIEU"]], on="TEN_NM", how="left")
merged_df

,DVDL,NMTD,TEN_NM,MADIEMDO,TD_THAMCHIEU
0,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ TÀ LÀNG,TA_LANG,G2A121S000D371,TUYEN_QUANG
1,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ TÀ LÀNG,TA_LANG,G2A121S000M371,TUYEN_QUANG
2,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ THƯỢNG ÂN,THUONG_AN,G2A122S000D371,TUYEN_QUANG
3,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ THƯỢNG ÂN,THUONG_AN,G2A122S000M371,TUYEN_QUANG
4,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ NẬM CẮT,NAM_CAT,G2A123S000M371,TUYEN_QUANG
...,...,...,...,...,...
804,CTY ĐIỆN LỰC TUYÊN QUANG,NMTĐ Sông Lô 8A,SONG_LO_8A,G2A203S000D131,TUYEN_QUANG
805,CTY ĐIỆN LỰC TUYÊN QUANG,NMTĐ Sông Lô 8B,SONG_LO_8B,G2A214S000D132,TUYEN_QUANG
806,CTY ĐIỆN LỰC TUYÊN QUANG,NMTĐ Sông Lô 8B,SONG_LO_8B,G2A214S000M132,TUYEN_QUANG
807,CTY ĐIỆN LỰC TUYÊN QUANG,NMTĐ Sông Lô 8B,SONG_LO_8B,G2A214S000D131,TUYEN_QUANG


In [ ]:
# Merge sản lượng từ file sản lượng ipp vào File thông tin NMTD với cột key là 'MADIEMDO'
merged_df_1 = merged_df.merge(new_data, on="MADIEMDO", how="left")
merged_df_1

,DVDL,NMTD,TEN_NM,MADIEMDO,TD_THAMCHIEU,NGAY,00:00:00,00:30:00,01:00:00,01:30:00,...,19:00:00,19:30:00,20:00:00,20:30:00,21:00:00,21:30:00,22:00:00,22:30:00,23:00:00,23:30:00
0,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ TÀ LÀNG,TA_LANG,G2A121S000D371,TUYEN_QUANG,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ TÀ LÀNG,TA_LANG,G2A121S000M371,TUYEN_QUANG,2024-01-01,0.0,0.0,0.0,0.0,...,2394.0,831.68,332.08,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ TÀ LÀNG,TA_LANG,G2A121S000M371,TUYEN_QUANG,2024-01-02,0.0,0.0,0.0,0.0,...,2058.0,754.40,343.84,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ TÀ LÀNG,TA_LANG,G2A121S000M371,TUYEN_QUANG,2024-01-03,0.0,0.0,0.0,0.0,...,1879.9,712.94,1.12,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ TÀ LÀNG,TA_LANG,G2A121S000M371,TUYEN_QUANG,2024-01-04,0.0,0.0,0.0,0.0,...,2076.5,1163.10,427.34,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77883,CTY ĐIỆN LỰC TUYÊN QUANG,NMTĐ Sông Lô 8B,SONG_LO_8B,G2A214S000M131,TUYEN_QUANG,2024-08-04,64.0,64.0,64.0,63.0,...,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
77884,CTY ĐIỆN LỰC TUYÊN QUANG,NMTĐ Sông Lô 8B,SONG_LO_8B,G2A214S000M131,TUYEN_QUANG,2024-08-05,0.0,0.0,0.0,0.0,...,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
77885,CTY ĐIỆN LỰC TUYÊN QUANG,NMTĐ Sông Lô 8B,SONG_LO_8B,G2A214S000M131,TUYEN_QUANG,2024-08-06,0.0,0.0,0.0,0.0,...,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
77886,CTY ĐIỆN LỰC TUYÊN QUANG,NMTĐ Sông Lô 8B,SONG_LO_8B,G2A214S000M131,TUYEN_QUANG,2024-08-07,0.0,0.0,0.0,0.0,...,17.0,42.00,40.00,39.0,38.0,38.0,37.0,35.0,36.0,35.0


In [ ]:
# xoá những dòng nào mà tất cả giá trị từ 00:00:00 đến 23:30:00 đều blank
merged_df_2 = merged_df_1.dropna(subset=merged_df_1.columns[6:], how="all")

In [ ]:
# Export ra file excel
merged_df_2.to_excel(r"C:\Khue\TDN\data\processed\2024.xlsx", index=False)

In [10]:
import os
import pandas as pd

# Đường dẫn thư mục chứa file
folder_path = r"C:\Khue\TDN\data\processed"

# Danh sách tên file bạn muốn đọc (chỉ 4 file của các năm)
file_names = ["2021.xlsx", "2022.xlsx", "2023.xlsx", "2024.xlsx"]

# Tạo đường dẫn đầy đủ cho từng file
files = [os.path.join(folder_path, file) for file in file_names]

# Đọc từng file vào DataFrame
dfs = [pd.read_excel(file) for file in files]

# Gộp các DataFrame lại với nhau theo chiều dọc
merged_df = pd.concat(dfs, ignore_index=True)

# Lưu DataFrame đã gộp ra file Excel mới trong cùng thư mục
output_file = os.path.join(folder_path, "merged_data.xlsx")
merged_df.to_excel(output_file, index=False)

print(f"Đã gộp {len(files)} file thành công vào file: {output_file}")


Đã gộp 4 file thành công vào file: C:\Khue\TDN\data\processed\merged_data.xlsx


In [13]:
# Chuyển cột thời gian 'START_TIME' sang kiểu datetime để sắp xếp chính xác
merged_df['NGAY'] = pd.to_datetime(merged_df['NGAY'], errors='coerce')

# Sắp xếp dữ liệu trong từng nhóm 'MADIEMDO' theo 'START_TIME'
df_sorted = merged_df.groupby('MADIEMDO', group_keys=False).apply(lambda x: x.sort_values(by='NGAY'))

# Lưu lại dữ liệu đã sắp xếp vào file mới
output_file = "sanluongipp_sorted.xlsx"
df_sorted.to_excel(output_file, index=False)

print(f"Đã sắp xếp dữ liệu theo 'MADIEMDO' và 'START_TIME', lưu vào: {output_file}")


Đã sắp xếp dữ liệu theo 'MADIEMDO' và 'START_TIME', lưu vào: sanluongipp_sorted.xlsx
